In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.graph_objects as go
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import math for math.ceil
import math

# Importing middleware for python glue
from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = 'aacuser'
password = 'Winter#123'
# The necessary fields for connection to the AAC database
HOST = 'nv-desktop-services.apporto.com'
PORT = 32603
DB = 'AAC'
COL = 'animals'

# Connect to database via CRUD Module
db = AnimalShelter(username, password, HOST, PORT, DB, COL)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))
prev_radio_val = ""

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

# Queries for filtering the data table based on user interaction
# Water Rescue query
wr_query = {
    "animal_type" : "Dog",
    "breed" : {"$in" : ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
    "sex_upon_outcome" : "Intact Female",
    "age_upon_outcome_in_weeks" : {
        "$gte" : 26,
        "$lte" : 156
    }
}

# Mountain / Wilderness query
mw_query = {
    "animal_type" : "Dog",
    "breed" : {"$in" : ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
    "sex_upon_outcome" : "Intact Male",
    "age_upon_outcome_in_weeks" : {
        "$gte" : 26,
        "$lte" : 156
    }
}

# Disaster / Individual query
di_query = {
    "animal_type" : "Dog",
    "breed" : {"$in" : ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
    "sex_upon_outcome" : "Intact Male",
    "age_upon_outcome_in_weeks" : {
        "$gte" : 20,
        "$lte" : 300
    }
}

# Reset query
reset_query = {}


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

PAGE_SIZE = 15

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    # Created a Div class to display the Image, Title and my Unique ID in a single row
    html.Div(children=[
        html.A(children=html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), height=150),
               href='https://www.snhu.edu'), 
        html.B(children=html.H1('CS-340 Grazioso Salvare'), style={'margin' : 'auto'}),
        html.A(children='Created by Aaron McDonald.', href='https://github.com/Origier')],
             style={'display' : 'flex'}
    ),
    html.Hr(),
    # Radio Buttons for filtering the data table
    dcc.RadioItems(options=[
        {
            "label":
            [
                html.Span("Water Rescue", style={'padding-right': 10})
            ],
            "value": "WR",
        },
        {
            "label":
            [
                html.Span("Mountain / Wilderness", style={'padding-right': 10})
            ],
            "value": "MW",
        },
        {
            "label":
            [
                html.Span("Disaster / Individual", style={'padding-right': 10})
            ],
            "value": "DI",
        },
        {
            "label":
            [
                html.Span("Reset", style={'padding-right': 10})
            ],
            "value": "R",
        },
        ],
        value='R',
        inline=True,
        id="RadioOptions"),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        # data=df.to_dict('records'), Removing for backend pagination
        # Limiting the page size to 15
        page_size=PAGE_SIZE,
        # Setting the current page
        page_current=0,
        # Setting the page action to custom to allow for backend pagination
        page_action='custom',
        # Setting the page count according to the size of the data we pulled
        page_count=math.ceil(len(df) / PAGE_SIZE),
        
        # Adding sorting options to allow for sorting with pagination
        sort_action='custom',
        sort_mode='single',
        sort_by=[],
        # Added rows selectable to work with the geolocation table
        row_selectable='single',
        selected_rows = [0]
                        ),
    html.Br(),
    html.Hr(),
    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
            style={'width' : '100%'}
            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            style={'width' : '100%'}
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
    
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_rows')]
)
def update_styles(selected_rows):
    return [{
        'if': { 'row_index': i },
        'background_color': '#D2F3FF'
    } for i in selected_rows]

# Callback function to perform all of the data manipulation and visuals
# Handles sorting, backend pagination, radio filtering and graphing
@app.callback(
    Output('datatable-id', 'data'),
    Output('datatable-id', 'selected_rows'),
    Output('datatable-id', 'page_count'),
    Output('graph-id', "children"),
    Input('datatable-id', 'page_current'),
    Input('datatable-id', 'page_size'),
    Input('datatable-id', 'sort_by'),
    Input('RadioOptions', 'value')
)
def update_table(page_current, page_size, sort_by, value):
    global prev_radio_val
    global df
    # Running the query only when needed
    if prev_radio_val != value:
        # Determining which query to run based on the radio buttons
        if (value == "WR"):
            query = wr_query
        elif (value == "MW"):
            query = mw_query
        elif (value == "DI"):
            query = di_query
        else:
            query = reset_query
    
        prev_radio_val = value
    
        # Using the query to pull data from the database
        df = pd.DataFrame.from_records(db.read(query))
        df.drop(columns=['_id'],inplace=True)

    # Calculating the new page count
    page_count=math.ceil(len(df) / PAGE_SIZE)
    
    # Performing the sorting on the dataset for backend pagination
    if len(sort_by):
        dff = df.sort_values(
            sort_by[0]['column_id'],
            ascending=sort_by[0]['direction'] == 'asc',
            inplace=False
        )
    else:
        dff = df
    
    # Generating the data slice to be sent to the view
    data = dff.iloc[
        page_current * page_size : (page_current + 1) * page_size
    ].to_dict('records')
    
    # Turning the selected dataset into a dictionary for the pie chart
    df_data = df.to_dict('records')
      
    # Selecting all of the breeds
    breeds = []
    for record in df_data:
        for key in record.keys():
            if key == "breed":
                breeds.append(record[key])
    # All of the distinct breeds
    breed_labels = set(breeds)
    # Counting the number of the different breeds
    breed_count = {}
    for breed in breed_labels:
        for other_breed in breeds:
            if breed == other_breed:
                breed_count[breed] = breed_count.get(breed, 0) + 1
    # Threshold for the "other" category - keeping it to 1 percent
    threshold = len(df_data) * 0.01
    # Calculating the "other" category and creating the separate labels
    other_value = 0
    main_values = []
    main_labels = []
    for breed in breed_count.keys():
        if breed_count[breed] < threshold:
            other_value += breed_count[breed]
        else:
            main_labels.append(breed)
            main_values.append(breed_count[breed])
    # If there is an "other" category then add it to the pie chart        
    if (other_value > 0):
        main_labels.append("Other")
        main_values.append(other_value)
    # Creating the go.Pie chart     
    pie_chart = figure = go.Figure(
        data=[go.Pie(labels=main_labels, 
                    values=main_values, 
                    name='breed', 
                    title='Preferred Animals', 
                    hole=.3)])
                    
    # Updating the legend to be more readable
    pie_chart.update_layout(
                    legend=dict(
                    orientation="v",
                    yanchor="top",
                    y=1,
                    xanchor="right",
                    x=0))
    
    # Creating the graph dash object that contains the chart
    graph = [dcc.Graph(figure = pie_chart)]
    
    # Returns the data, selected row, newly created page count, and visual graph
    return data, [0], page_count, graph


# This callback will update the geo-location chart for the selected data entry
# the derived virtual data and rows have been replaced with the actual data and selected rows.
# This is to accommodate for the backend pagination.
# For this application, we are only permitting single row selection so there is only one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "data"),
     Input('datatable-id', "selected_rows")])
def update_map(viewData, index):    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can 
    # be converted to a row index here
    if index is None:
        #row = 0
        pass
    else: 
        row = index[0]

    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
           center=[30.75,-97.48], zoom=10, children=[
           dl.TileLayer(id="base-layer-id"),
           # Marker with tool tip and popup
           # Column 13 and 14 define the grid-coordinates for 
           # the map
           # Column 4 defines the breed for the animal
           # Column 9 defines the name of the animal
           dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]],
              children=[
              dl.Tooltip(dff.iloc[row,4]),
              dl.Popup([
                 html.H1("Animal Name"),
                html.P(dff.iloc[row,9])
             ])
          ])
       ])
    ]
    

app.run_server(debug=True)


Dash app running on http://127.0.0.1:12185/
